# 并查集 (Disjoint Set Union - DSU) 详解

## 1. 什么是并查集?

并查集是一种树形的数据结构，用于处理一些不相交集合（Disjoint Sets）的合并及查询问题。它主要支持两种操作：

- **Find (查找)**: 确定一个元素属于哪个集合。这通常通过查找集合的代表元素（根节点）来完成。
- **Union (合并)**: 将两个不相交的集合合并成一个集合。

这种数据结构在解决图论中的连通性问题、网络连接问题以及许多算法（如 Kruskal 算法求最小生成树）中非常有用。

## 2. 核心概念

我们可以用一个数组（或字典）来表示并查集，其中数组的索引代表元素，数组的值代表其父节点。

- **初始化**: 最初，每个元素都是一个独立的集合，其父节点就是它自己。`parent[i] = i`。
- **Find 操作**: 从给定元素开始，沿着父节点指针向上查找，直到找到根节点（即父节点是其自身的节点）。这个根节点就是该元素所在集合的代表。
- **Union 操作**: 要合并两个元素 `x` 和 `y` 所在的集合，我们首先通过 `find` 操作找到它们各自的根节点 `rootX` 和 `rootY`。如果它们不相同，就将一个根节点的父指针指向另一个根节点，从而将两棵树合并为一棵。

## 3. 基本实现

In [2]:
class DisjointSetBasic:
    def __init__(self, n):
        # 初始化，每个元素的父节点都是自己
        self.parent = list(range(n))

    def find(self, i):
        # 查找根节点
        if self.parent[i] == i:
            return i
        # 递归查找父节点的根节点
        return self.find(self.parent[i])

    def union(self, i, j):
        # 合并两个元素所在的集合
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i != root_j:
            self.parent[root_i] = root_j

# 示例
ds = DisjointSetBasic(5)
print(f"初始父节点: {ds.parent}")

ds.union(0, 1)
ds.union(2, 3)
ds.union(0, 4)

print(f"合并后的父节点: {ds.parent}")
print(f"元素 1 的根是: {ds.find(1)}")
print(f"元素 4 的根是: {ds.find(4)}")
print(f"元素 3 的根是: {ds.find(3)}")
print(ds.find(0))

初始父节点: [0, 1, 2, 3, 4]
合并后的父节点: [1, 4, 3, 3, 4]
元素 1 的根是: 4
元素 4 的根是: 4
元素 3 的根是: 3
4


基本实现在处理大规模数据或形成长链时效率较低，`find` 操作可能退化成 O(n)。为了提高效率，我们需要进行优化。

## 4. 优化方法

### a) 路径压缩 (Path Compression)

在 `find` 操作中，当我们查找到根节点后，将查找路径上的所有节点的父指针直接指向根节点。这样可以极大地扁平化树的结构，使得后续的 `find` 操作非常快。

### b) 按秩合并 (Union by Rank) 或 按大小合并 (Union by Size)

为了避免合并时产生过深的树，我们引入一个额外的 `rank` 或 `size` 数组。
- **按秩合并**: `rank` 记录树的深度。合并时，总是将深度较浅的树连接到深度较深的树的根节点上。
- **按大小合并**: `size` 记录每个集合中元素的数量。合并时，总是将元素数量较少的集合连接到元素数量较多的集合的根节点上。

这两种方法都能有效地保持树的平衡。实践中，按大小合并更简单且同样高效。

## 5. 优化后的实现 (路径压缩 + 按大小合并)

In [ ]:
class DisjointSetOptimized:
    def __init__(self, n):
        self.parent = list(range(n))
        # 初始化每个集合的大小为 1
        self.size = [1] * n

    def find(self, i):
        # 查找根节点，并进行路径压缩
        if self.parent[i] == i:
            return i
        # 递归查找，并将路径上所有节点直接指向根节点
        self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        # 按大小合并两个集合
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i != root_j:
            # 将小树合并到大树
            if self.size[root_i] < self.size[root_j]:
                root_i, root_j = root_j, root_i  # 保证 root_i 是较大的树
            self.parent[root_j] = root_i
            self.size[root_i] += self.size[root_j]

# 示例
ds_opt = DisjointSetOptimized(10)
print(f"初始父节点: {ds_opt.parent}")
print(f"初始大小:   {ds_opt.size}")

ds_opt.union(0, 1)
ds_opt.union(2, 3)
ds_opt.union(0, 2)
ds_opt.union(5, 6)
ds_opt.union(7, 8)
ds_opt.union(5, 7)
ds_opt.union(0, 5)

print(f"合并后父节点: {ds_opt.parent}")
print(f"合并后大小:   {ds_opt.size}")

# 执行一次 find 来观察路径压缩的效果
print(f"查找元素 3 的根: {ds_opt.find(3)}")
print(f"路径压缩后父节点: {ds_opt.parent}")

In [2]:
x = bin(1)
res = 0
temp = 0
for i in str(x):
    if i == "1":
        temp+=1
        res = max(res, temp)
    else:
        temp=0

In [1]:
from collections import defaultdict
 
# 处理输入
n, m = map(int, input().split())
n //= 10  # 价格总为 10 的倍数，优化空间复杂度
prices = defaultdict(lambda: [0, 0, 0])  # 主从物品的价格
prices

defaultdict(<function __main__.<lambda>()>, {})

In [ ]:
import sys
from collections import defaultdict

def main():
    data = [
    [800, 2, 0],  # 主件
    [400, 5, 1],  # 附件，属于主件1
    [300, 5, 1],  # 附件，属于主件1
    [400, 3, 0],  # 主件
    [500, 2, 0]   # 主件
]
    if not data:
        return
    
    n = int(data[0])
    m = int(data[1])
    n //= 10
    items = []
    index = 2
    for i in range(m):
        v = int(data[index])
        w = int(data[index+1])
        q = int(data[index+2])
        index += 3
        v //= 10
        items.append((v, w, q))
        
    main_items = []
    attachments = defaultdict(list)
    
    for i in range(m):
        v, w, q = items[i]
        if q == 0:
            main_items.append((v, w, i+1))
        else:
            attachments[q].append((v, w))
            
    groups = []
    for item in main_items:
        v0, w0, idx = item
        att_list = attachments.get(idx, [])
        options = []
        options.append((v0, v0 * w0))
        if len(att_list) >= 1:
            v1, w1 = att_list[0]
            options.append((v0 + v1, v0*w0 + v1*w1))
        if len(att_list) >= 2:
            v2, w2 = att_list[1]
            options.append((v0 + v2, v0*w0 + v2*w2))
            options.append((v0 + v1 + v2, v0*w0 + v1*w1 + v2*w2))
        groups.append(options)
        
    dp = [0] * (n + 1)
    for group in groups:
        for j in range(n, -1, -1):
            for c, s in group:
                if j >= c:
                    if dp[j] < dp[j - c] + s:
                        dp[j] = dp[j - c] + s
    ans = max(dp) * 10
    print(ans)
    
main()

## 6. 时间复杂度

同时使用路径压缩和按秩/大小合并后，并查集的 `find` 和 `union` 操作的平均时间复杂度接近 O(1)，严格来说是 O(α(n))，其中 α(n) 是反阿克曼函数，其增长速度非常慢，对于所有实际问题，其值可以看作是一个不超过 5 的常数。

## 7. 应用场景

1.  **检测无向图中的环**: 遍历图的每条边 (u, v)。对每个边，检查 u 和 v 是否已在同一集合中。如果是，则说明添加这条边会形成一个环。如果不是，则合并它们。
2.  **最小生成树 (MST)**: 在 Kruskal 算法中，用于判断新加入的边是否会与已选择的边形成环。
3.  **计算图的连通分量**: 遍历所有节点，如果两个节点在同一个集合中，它们就属于同一个连通分量。集合的数量就是连通分量的数量。
4.  **网络连接问题**: 判断网络中的两个节点是否相连。